# **載入套件**

In [1]:
from __future__ import unicode_literals, print_function, division
import numpy as np
import sys
import math
import csv


activate_codes_num = -1
next_k_step = 1
training_chunk = 0
test_chunk = 1

# **生成字典與計算最終向量維度**

In [2]:
def generate_dictionary_BA(files, attributes_list):
    # path = '../Minnemudac/'
    #files = ['Coborn_history_order.csv','Coborn_future_order.csv']
    #files = ['BA_history_order.csv', 'BA_future_order.csv']
    #attributes_list = ['MATERIAL_NUMBER']

    #定義兩個字典，用於儲存屬性值和它們相對應的索引
    dictionary_table = {}
    counter_table = {}

    for attr in attributes_list:
        dictionary = {}
        dictionary_table[attr] = dictionary
        counter_table[attr] = 0

    csv.field_size_limit(300) # 設置 CSV 讀取的最大長度

    for filename in files:
        count = 0 #用於追蹤當前文件中處理的行數
        with open(filename, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='|') # 指定逗號（,）為字段分隔符，並將豎線（|）作為引號字符
            for row in reader:
                if count == 0: # 跳過第一行（標題行）:
                    count += 1
                    continue
                key = attributes_list[0] # 取出第一個屬性，即 MATERIAL_NUMBER
                if row[3] not in dictionary_table[key]: # 如果 row[2] 的值還沒有在字典中，則將其添加到字典中，並賦予一個索引
                    dictionary_table[key][row[3]] = counter_table[key]
                    counter_table[key] = counter_table[key] + 1 # 更新 counter_table 中對應屬性的計數
                    count += 1

    print("Conter Table : ", counter_table)
    print("Dictionary Table : " , dictionary_table)

    total = 0
    for key in counter_table.keys():
        total = total + counter_table[key]

    print('# dimensions of final vector: ' + str(total) + ' | '+str(count-1))

    return dictionary_table, total, counter_table


# 從 CSV **文件中讀取數據並處理，將其轉換為特定格式的數據結構**

In [3]:
# 從 CSV 文件中讀取數據並處理，將其轉換為特定格式的數據結構
def read_claim2vector_embedding_file_no_vector(files): #
    #attributes_list = ['DRG', 'PROVCAT ', 'RVNU_CD', 'DIAG', 'PROC']
    attributes_list = ['MATERIAL_NUMBER'] # 一個包含要處理的屬性名稱的列表(存了商品編號）
    # path = '../Minnemudac/'
    print('start dictionary generation...')
    dictionary_table, num_dim, counter_table = generate_dictionary_BA(files, attributes_list) # 生成字典並返回三個值 #files是./data/TaFang_history_NB.csv 跟 ./data/TaFang_future_NB.csv
    print('finish dictionary generation*****')
    usr_attr = 'CUSTOMER_ID'
    ord_attr = 'ORDER_NUMBER'
    freq_max = 200
    ## all the follow three ways array. First index is patient, second index is the time step, third is the feature vector
    data_chunk = [] # 用於儲存處理後數據的列表
    day_gap_counter = []
    claims_counter = 0
    num_claim = 0
    code_freq_at_first_claim = np.zeros(num_dim+2) #生成一個長度為 num_dim + 2 的零向量(?)

    #print("data_chunk:",data_chunk)

    # 迭代處理 files 列表中的每個文件
    for file_id in range(len(files)):

        count = 0
        data_chunk.append({})

        #print("data_chunk2:",data_chunk)

        filename = files[file_id] # 打開文件
        with open(filename, 'r') as csvfile:
            #gap_within_one_year = np.zeros(365)
            reader = csv.DictReader(csvfile) # 讀取 CSV 文件

            # 用於在下面的循環中跟踪上一行的數據
            last_pid_date = '*'
            last_pid = '-1'
            last_days = -1
            # 2 more elements in the end for start and end states
            feature_vector = []

            # 對 CSV 文件中的每一行進行迭代
            for row in reader:


                cur_pid_date = row[usr_attr] + '_' + row[ord_attr] # 從當前行生成一個唯一的標識符 (1_14)
                #print(cur_pid_date)
                cur_pid = row[usr_attr] # 並獲取用戶 ID (1)
                #cur_days = int(row[ord_attr])

                # 如果當前用戶 ID 不等於上一個用戶 ID，則在 data_chunk 中為這個新用戶創建一個新條目
                if cur_pid != last_pid:
                    # start state
                    tmp = [-1]
                    data_chunk[file_id][cur_pid] = []
                    data_chunk[file_id][cur_pid].append(tmp)

                    #print(data_chunk[file_id][cur_pid])

                # 如果當前的 cur_pid_date 不等於 last_pid_date，則對 feature_vector 進行排序並將其添加到 data_chunk 中的相應位置，然後重置 feature_vector
                if cur_pid_date not in last_pid_date:
                    if last_pid_date not in '*' and last_pid not in '-1':
                        sorted_feature_vector = np.sort(feature_vector)

                        #print("sorted_feature_vector: ",sorted_feature_vector)

                        data_chunk[file_id][last_pid].append(sorted_feature_vector)
                        if len(sorted_feature_vector) > 0:
                            count = count + 1
                        #data_chunk[file_id][last_pid].append(feature_vector)
                    feature_vector = []

                    claims_counter = 0
                # 如果當前用戶 ID 不等於上一個用戶 ID，則在 data_chunk 中的上一個用戶 ID 下添加一個結束標記
                if cur_pid != last_pid:
                    # end state
                    if last_pid not in '-1':

                        tmp = [-1]
                        data_chunk[file_id][last_pid].append(tmp)
                        #print(data_chunk[file_id][last_pid])

                # --- 計算當前行的特徵索引 idx ---
                key = attributes_list[0]

                within_idx = dictionary_table[key][row[key]]
                previous_idx = 0

                for j in range(attributes_list.index(key)):
                    previous_idx = previous_idx + counter_table[attributes_list[j]]
                idx = within_idx + previous_idx
                #print("idx :", idx)
                # --- 計算當前行的特徵索引 idx ---

                # 如果 idx 不在 feature_vector 中，則將其添加到 feature_vector 中
                # 針對每個 CUSTOMER_ID 與 ORDER_NUMBER 組合去儲存特徵向量
                # set corresponding dimention to 1
                if idx not in feature_vector:
                    feature_vector.append(idx)

                # 更新 last_pid_date 和 last_pid 值
                last_pid_date = cur_pid_date
                last_pid = cur_pid
                #last_days = cur_days
                if file_id == 1:
                    claims_counter = claims_counter + 1

            # 在循環結束後，將最後一個 feature_vector 添加到 data_chunk 中
            if last_pid_date not in '*' and last_pid not in '-1':
                data_chunk[file_id][last_pid].append(np.sort(feature_vector))
                #print(data_chunk[file_id][last_pid])

    print("num_dim+2 :", num_dim + 2 ,"code_freq_at_first_claim :",code_freq_at_first_claim)

    #把 data_chunk 儲存查看內容
    # import pickle
    # with open('data_chunk.pkl', 'wb') as file:
    #     pickle.dump(data_chunk, file)
    # file = "data_chunk.pkl"
    # f = open(file,'rb')
    # data = pickle.load(f)

    # print(data[0]["2"])
    # print(data[1]["2"])
    # #print(len(data))

    return data_chunk, num_dim + 2, code_freq_at_first_claim # 返回處理後的數據

# **分割資料集**

In [4]:
def partition_the_data_validate(data_chunk, key_set, next_k_step):

    filtered_key_set = [] # 用於儲存經過篩選後的 key
    past_chunk = 0
    future_chunk = 1

    for key in key_set:
        if len(data_chunk[past_chunk][key]) <= 7: # 這裡的值要隨資料集不同去修改，Tafeng 3 Dunn 7
            continue
        if len(data_chunk[future_chunk][key]) < 2 + next_k_step:
            continue
        filtered_key_set.append(key)

    training_key_set = filtered_key_set[0:int(4 / 5 * len(filtered_key_set)*0.9)]
    validation_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)*0.9):int(4 / 5 * len(filtered_key_set))]
    test_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)):]

    print('Number of training instances: ' + str(len(training_key_set)))
    print('Number of validation instances: ' + str(len(validation_key_set)))
    print('Number of test instances: ' + str(len(test_key_set)))

    print('Training Key Set: ',training_key_set)

    return training_key_set, validation_key_set, test_key_set

# 分群計算，獲得所有群組的平均向量

**將一系列的歷史數據向量按給定的群組大小分組，並計算每個群組中的向量的平均值**

假設 his_list 長度為 10，group_size 為 3，\
est_num_vec_each_block = 10 / 3 ≈ 3.33（每個群組中平均應有大約 3.33 個向量）\
base_num_vec_each_block = 3.33 向下取整 = 3（每個群組至少應有 3 個向量）\
residual = 3.33 - 3 = 0.33 （差異）\
num_vec_has_extra_vec = 1（有 1 個群組將比其他群組多一個向量）

In [5]:
def group_history_list(his_list,group_size):
    grouped_vec_list = []
    if len(his_list) < group_size:
        #sum = np.zeros(len(his_list[0]))
        for j in range(len(his_list)):
            grouped_vec_list.append(his_list[j])

        return grouped_vec_list, len(his_list)
    else:
        est_num_vec_each_block = len(his_list)/group_size
        base_num_vec_each_block = int(np.floor(len(his_list)/group_size))
        residual = est_num_vec_each_block - base_num_vec_each_block

        num_vec_has_extra_vec = int(np.round(residual * group_size))

        if residual == 0:
            for i in range(group_size):
                if len(his_list)<1:
                    print('len(his_list)<1')
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                grouped_vec_list.append(sum/base_num_vec_each_block)
        else:

            for i in range(group_size - num_vec_has_extra_vec):
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*base_num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                    last_idx = i * base_num_vec_each_block + j
                grouped_vec_list.append(sum/base_num_vec_each_block)

            est_num = int(np.ceil(est_num_vec_each_block))
            start_group_idx = group_size - num_vec_has_extra_vec

            if len(his_list) - start_group_idx*base_num_vec_each_block >= est_num_vec_each_block:
                for i in range(start_group_idx,group_size):
                    sum = np.zeros(len(his_list[0]))
                    for j in range(est_num):
                        # if residual+(i-1)*est_num_vec_each_block+j >= len(his_list):
                        #     print('residual+(i-1)*num_vec_each_block+j')
                        #     print('len(his_list)')
                        iidxx = last_idx + 1+(i-start_group_idx)*est_num+j
                        if  iidxx >= len(his_list) or iidxx<0:
                            print('last_idx + 1+(i-start_group_idx)*est_num+j')
                        sum += his_list[iidxx]
                    grouped_vec_list.append(sum/est_num)
        return grouped_vec_list, group_size

# 獲得用戶歷史向量

**時間衰減計算**

**對於每次交易中所購買的項目位置，會根據時間衰減權重賦予相應的值，而其他位置則保持為0。**

例如：對於一個具有15筆訓練資料的用戶，他針對每一筆去計算出時間衰減權重，之後對於每一筆資料中所購買的項目位置，假設購買項目1、3、5，則在這三個位置上賦予時間衰減權重值，而其餘部分則保持為0，使得為每次交易創造一個長度為 output_size 的有零也有值的新向量。

...



In [6]:
# data_chunk[training_chunk],training_key_set, input_size,group_size, within_decay_rate,group_decay_rate
# input_size 就是 num_dim + 2 （10997+2）

def temporal_decay_sum_history(data_set, key_set, output_size,group_size,within_decay_rate,group_decay_rate):
    sum_history = {}

    for key in key_set:

        vec_list = data_set[key]
        num_vec = len(vec_list) - 2

        # User 2 Output :
        # vec_list: [[-1], array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]) ... array([ 52, 106, 142, 143, 144, 145, 146, 147]), [-1]]
        # num_vec: 15（購物籃數量）（-2 是減去前後的 [-1] ）

        his_list = []

        for idx in range(1,num_vec+1):
            his_vec = np.zeros(output_size)
            decayed_val = np.power(within_decay_rate,num_vec-idx)
            for ele in vec_list[idx]:
                his_vec[ele] = decayed_val
            his_list.append(his_vec) # 包含了用戶所有交易的時間衰減向量

        grouped_list,real_group_size = group_history_list(his_list,group_size)
        his_vec = np.zeros(output_size)
        for idx in range(real_group_size):
            decayed_val = np.power(group_decay_rate, group_size - 1 - idx)
            if idx>=len(grouped_list):
                print( 'idx: '+ str(idx))
                print('len(grouped_list): ' + str(len(grouped_list)))
            his_vec += grouped_list[idx]*decayed_val
        sum_history[key] = his_vec/real_group_size
    return sum_history

# 合併自己向量與鄰居向量

針對測試集中的用戶，考慮自己向量與鄰居向量之比例

In [7]:
# 傳入訓練集與測試集的用戶向量、訓練集與測試集、KNN 結果的索引、合併參數 alpha

def merge_history(sum_history_test,test_key_set,training_sum_history_test,training_key_set,index,alpha):

    merged_history = {}

    for test_key_id in range(len(test_key_set)):
        test_key = test_key_set[test_key_id]
        test_history = sum_history_test[test_key]

        sum_training_history = np.zeros(len(test_history))

        for indecis in index[test_key_id]:
            training_key = training_key_set[indecis]
            sum_training_history += training_sum_history_test[training_key]

        sum_training_history = sum_training_history/len(index[test_key_id])
        merge = test_history*alpha + sum_training_history*(1-alpha)
        merged_history[test_key] = merge

    return merged_history

# KNN 尋找相似鄰居

對測試集數據行最近鄰居搜索，返回每個測試點的最近鄰居索引（indices）和相應的距離（distances）

In [8]:
from sklearn.neighbors import NearestNeighbors

def KNN(query_set, target_set, k):

    history_mat = []
    for key in target_set.keys():
        history_mat.append(target_set[key])
    test_mat = []
    for key in query_set.keys():
        test_mat.append(query_set[key])

    #print('Finding k nearest neighbors...')
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute').fit(history_mat)
    distances, indices = nbrs.kneighbors(test_mat)
    #print('Finish KNN search.' )

    #print('Index:',indices)
    #print('Distances:',distances)
    return indices,distances

# Precision、recall、Fscore


In [9]:
def get_precision_recall_Fscore(groundtruth,pred):
    a = groundtruth
    b = pred
    correct = 0
    truth = 0
    positive = 0

    for idx in range(len(a)):
        if a[idx] == 1:
            truth += 1
            if b[idx] == 1:
                correct += 1
        if b[idx] == 1:
            positive += 1

    flag = 0
    if 0 == positive:
        precision = 0
        flag = 1
        #print('postivie is 0')
    else:
        precision = correct/positive
    if 0 == truth:
        recall = 0
        flag = 1
        #print('recall is 0')
    else:
        recall = correct/truth

    if flag == 0 and precision + recall > 0:
        F = 2*precision*recall/(precision+recall)
    else:
        F = 0
    return precision, recall, F, correct

# evaluate

In [10]:
def predict_with_elements_in_input(sum_history,key):
    # 傳入：合併後的測試集用戶向量字典和指定用戶的 Key 值
    output_vectors = [] # 空列表，用於儲存生成的預測向量

    # next_k_step 預設為 1
    for idx in range(next_k_step): # 指定要生成多少個預測向量
        vec = sum_history[key] # 獲得與 key 相關聯的合併歷史向量
        output_vectors.append(vec)
        # print(output_vectors)
    return output_vectors

In [11]:
def get_HT(groundtruth, pred_rank_list,k):
    count = 0
    for pred in pred_rank_list:
        if count >= k:
            break
        if groundtruth[pred] == 1:
            return 1
        count += 1

    return 0

In [12]:
def get_NDCG1(groundtruth, pred_rank_list,k):
    count = 0
    dcg = 0
    for pred in pred_rank_list:
        if count >= k:
            break
        if groundtruth[pred] == 1:
            dcg += (1)/math.log2(count+1+1)
        count += 1
    idcg = 0
    num_real_item = np.sum(groundtruth)
    num_item = int(num_real_item)
    for i in range(num_item):
        idcg += (1) / math.log2(i + 1 + 1)
    ndcg = dcg / idcg
    return ndcg

In [13]:
def evaluate(data_chunk,  training_key_set, test_key_set, input_size, group_size,
             within_decay_rate, group_decay_rate, num_nearest_neighbors, alpha,  topk):
    activate_codes_num = -1
    temporal_decay_sum_history_training = temporal_decay_sum_history(data_chunk[training_chunk],
                                                                     training_key_set, input_size,
                                                                     group_size, within_decay_rate,
                                                                     group_decay_rate)
    temporal_decay_sum_history_test = temporal_decay_sum_history(data_chunk[training_chunk],
                                                                 test_key_set, input_size,
                                                                 group_size, within_decay_rate,
                                                                 group_decay_rate)


    index, distance = KNN(temporal_decay_sum_history_test, temporal_decay_sum_history_training,
                          num_nearest_neighbors)

    # 傳入測試集和訓練集歷史用戶向量、相應的鍵集、KNN 結果的索引，以及一個合併參數 alpha
    sum_history = merge_history(temporal_decay_sum_history_test, test_key_set, temporal_decay_sum_history_training,
                                training_key_set, index, alpha)


    if activate_codes_num < 0:
        prec = []
        rec = []
        F = []
        prec1 = []
        rec1 = []
        F1 = []
        prec2 = []
        rec2 = []
        F2 = []
        prec3 = []
        rec3 = []
        F3 = []
        NDCG = []
        n_hit = 0

        num_ele = topk
        count = 0
        for iter in range(len(test_key_set)):
            input_variable = data_chunk[training_chunk][test_key_set[iter]]
            target_variable = data_chunk[test_chunk][test_key_set[iter]]

            if len(target_variable) < 2 + next_k_step:
                continue

            count += 1
            output_vectors = predict_with_elements_in_input(sum_history, test_key_set[iter])
            top = 400
            hit = 0

            for idx in range(len(output_vectors)):
                output = np.zeros(input_size)
                target_topi = output_vectors[idx].argsort()[::-1][:top]

                # print('top i :',target_topi)

                c = 0

                for i in range(top):
                    if c >= num_ele:
                        break
                    output[target_topi[i]] = 1
                    c += 1

                vectorized_target = np.zeros(input_size) # 創建目標向量

                for ii in target_variable[1 + idx]:
                    vectorized_target[ii] = 1

                precision, recall, Fscore, correct = get_precision_recall_Fscore \
                    (vectorized_target, output)
                prec.append(precision)
                rec.append(recall)
                F.append(Fscore)

                if idx == 0:
                    prec1.append(precision)
                    rec1.append(recall)
                    F1.append(Fscore)
                elif idx == 1:
                    prec2.append(precision)
                    rec2.append(recall)
                    F2.append(Fscore)
                elif idx == 2:
                    prec3.append(precision)
                    rec3.append(recall)
                    F3.append(Fscore)

                hit += get_HT(vectorized_target, target_topi, num_ele)
                ndcg = get_NDCG1(vectorized_target, target_topi, num_ele)
                NDCG.append(ndcg)
            if hit == next_k_step:
                n_hit += 1


        # print('average precision of ' + ': ' + str(np.mean(prec)) + ' with std: ' + str(np.std(prec)))
        avg_precision = np.mean(prec)
        avg_recall = np.mean(rec)
        avg_F1 = np.mean(F)
        avg_ndcg = np.mean(NDCG)

        # 輸出結果
        print('Average Precision: ', avg_precision)
        print('Average Recall: ', avg_recall)
        print('Average F1 Score: ', avg_F1)
        print('Average NDCG: ', avg_ndcg)

In [14]:
def main():

    files = ['./data/dunnhumby_history.csv', './data/dunnhumby_future.csv'] # ./data/TaFang_history_NB.csv 跟 ./data/TaFang_future_NB.csv（歷史與未來的兩個檔案）

    data_chunk, input_size, code_freq_at_first_claim = read_claim2vector_embedding_file_no_vector(files) # 讀取和處理兩個檔案

    training_key_set, validation_key_set, test_key_set = partition_the_data_validate(data_chunk, list(data_chunk[test_chunk]), 1) # 將數據分為訓練、驗證和測試集


    # 參數設定
    num_nearest_neighbors = 300
    within_decay_rate = 0.9
    group_decay_rate = 0.6
    alpha = 0.7
    group_size = 7
    topk = 65

    print('Num. of top: ', topk)

    # 傳入上述參數 ＋ data_chunk、訓練集、測試集
    evaluate(data_chunk, training_key_set, test_key_set, input_size,group_size, within_decay_rate, group_decay_rate,num_nearest_neighbors, alpha,  topk)
    sys.stdout.flush() # 強制刷新輸出緩衝區，確保所有被緩存的輸出內容都立即被寫入到目標位置

if __name__ == '__main__':
    main()

start dictionary generation...
Conter Table :  {'MATERIAL_NUMBER': 3003}
Dictionary Table :  {'MATERIAL_NUMBER': {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '51': 50, '52': 51, '53': 52, '54': 53, '55': 54, '56': 55, '57': 56, '58': 57, '59': 58, '60': 59, '61': 60, '62': 61, '63': 62, '64': 63, '65': 64, '66': 65, '67': 66, '68': 67, '69': 68, '70': 69, '71': 70, '72': 71, '73': 72, '74': 73, '75': 74, '76': 75, '77': 76, '78': 77, '79': 78, '80': 79, '81': 80, '82': 81, '83': 82, '84': 83, '85': 84, '86': 85, '87': 86, '88': 87, '89': 88, '90': 89, '91': 